In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
from astropy.table import Table
from astropy.wcs import WCS

from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii

import wget
import matplotlib.image as mpimg

from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm


def pull_LS_fits_one(sample,i,pixscale):
    
    ra = sample['RA'][i]
    dec = sample['DEC'][i]        
        
    image_url = "https://www.legacysurvey.org/viewer/cutout.fits?ra="+str(ra)+"&dec="+str(dec)+"&layer=ls-dr9&pixscale="+str(pixscale)
    
    filename = image_path = homedir+'/github/WISE_test_two/cutout_'+str(ra)+'_'+str(dec)+'.fits'
        
    if os.path.exists(filename):
        os.remove(filename) # if exists, remove it directly
        image = wget.download(image_url, out=filename) # download it to the specific path.
        #print(image)
    else:
        image = wget.download(image_url, out=filename)
    




def align_fits_one(sample,wise_dir,i,initial_pixscale,percentile1=.5,percentile2=99.5,cmap='viridis'):
    
    #ra, dec, galname
    ra = sample['RA'][i]
    dec = sample['DEC'][i]
    galname = sample['prefix'][i]
    
    #define the images and their respective header information
    
    try:
        #WISE
        image_path = homedir+'/github/'+str(wise_dir)+'/'+str(galname)+'-unwise-w3-1Comp-galfit-out.fits'
        image_WISE, h_WISE = fits.getdata(image_path,1,header=True)
        
        wcs_WISE = WCS(h_WISE)
    
        v1 = [scoreatpercentile(image_WISE,percentile1),
            scoreatpercentile(image_WISE,percentile1)]
        v2 = [scoreatpercentile(image_WISE,percentile2),
            scoreatpercentile(image_WISE,percentile2)]
    
        #LS    
        pull_LS_fits_one(vfmain,i,initial_pixscale)
        image_path = homedir+'/github/'+str(wise_dir)+'/cutout_'+str(ra)+'_'+str(dec)+'.fits'
        image_LS, h_LS = fits.getdata(image_path,header=True)
    
        #This loop will filter the scale value that aligns the images...
        #note: be sure that the range is large enough to accommodate many potential pixscale values
        
        for scale in np.arange(1,12.5,0.2):

            if (h_WISE['NAXIS1']*h_WISE['CD1_1']/(h_LS['NAXIS1']*h_LS['CD1_1']) > 0.90)&(h_WISE['NAXIS1']*h_WISE['CD1_1']/(h_LS['NAXIS1']*h_LS['CD1_1']) < 1.10):
                pixscales.append(scale)
                print('complete: pixscale',scale)
                print('ratio',h_WISE['NAXIS1']*h_WISE['CD1_1']/(h_LS['NAXIS1']*h_LS['CD1_1']))
                break
            else:
                #print('scale tried:',scale)
                #print('ratio',h_WISE['NAXIS1']*h_WISE['CD1_1']/(h_LS['NAXIS1']*h_LS['CD1_1']))
                pull_LS_fits_one(vfmain,i,scale)
                image_LS, h_LS = fits.getdata(image_path,header=True)
                
                
        wcs_LS = WCS(h_LS)
    
        r=image_LS[0]
        g=image_LS[1]
        b=image_LS[2]
        rgb_LS = np.dstack((r,g,b))
        image_LS = rgb_LS
    
        images = [image_WISE,image_LS]
        titles = ['WISE Image','Legacy Image']
        #wcs = [wcs_WISE, wcs_LS]
    
        #create loop to display images side-by-side (template from Rose's code)
    
        plt.figure(figsize=(14,6))
        plt.subplots_adjust(wspace=.0)
        for i,im in enumerate(images): 

        
            plt.xlabel('RA')
            if i == 0:
                plt.subplot(1,3,i+1,projection=wcs_WISE)
                plt.imshow(im,origin='lower',cmap=cmap,vmin=v1[i],vmax=v2[i])
                plt.ylabel('DEC')
            if i == 1:
                plt.subplot(1,3,i+1,projection=wcs_LS[1])
                plt.imshow(im,origin='lower')
                ax = plt.gca()
                ax.set_yticks([])
        
            plt.title(titles[i],fontsize=16)
            
            #print('ratio',h_WISE['NAXIS1']*h_WISE['CD1_1']/(h_LS['NAXIS1']*h_LS['CD1_1']))
    except:
        pixscales.append(-999)
        pass


    homedir = os.getenv("HOME")

    vfmain = Table.read(homedir+'/github/research/sample_main_two.fits')

    #downloaded files will appear in this directory
    os.chdir(homedir+'/github/WISE_test_two')
   
   
   
   
    pixscales = []
    
    for i in range(0,len(vfmain)):
        
        print(i)
        align_fits_one(vfmain,i,1)
        filename = homedir+'/github/WISE_test_two/'+str(vfmain['VFID'][i])+'-mosaic.png'
        
        if os.path.exists(filename):
            os.remove(filename) # if exists, remove it directly
            plt.savefig('%s-mosaic.png'%vfmain['VFID'][i])
        else:
            plt.savefig('%s-mosaic.png'%vfmain['VFID'][i])

        plt.close()
    print(pixscales)
    
    pixscales_rounded = []
    for number in pixscales:
        pixscales_rounded.append(round(number,1))

In [ ]:
def ls_cutout_scaled(sample,i,pixscale_array):
    
    if pixscale[i] != -999:
            
        image_url = "https://www.legacysurvey.org/viewer/cutout.jpg?ra="+str(sample['RA'][i])+"&dec="+str(sample['DEC'][i])+"&layer=ls-dr9&pixscale="+str(pixscale[i])
        image = wget.download(image_url)
        #print(image)
        image_desired = "cutout_"+str(sample['prefix'][i])+".jpg"
        print(image_desired)
        os.rename(image,image_desired)
        
    else:
        image_url = "https://www.legacysurvey.org/viewer/cutout.jpg?ra="+str(sample['RA'][i])+"&dec="+str(sample['DEC'][i])+"&layer=ls-dr9&pixscale=5"
        image = wget.download(image_url)
        print(image)
        image_desired = "cutout_"+str(sample['prefix'][i])+".jpg"
        print(image_desired)
        os.rename(image,image_desired)

    homedir = os.getenv("HOME")
    #sample to scale...in the example below,sample_main_two.fits
    vfmain = Table.read(homedir+'/github/research/sample_main_two.fits')
    #downloaded files will appear in this directory
    os.chdir(homedir+'/github/WISE_test_two/ls_cutouts')
    
    
    for i in range(0,len(vfmain)):
        test3(vfmain,i,pixscale)